# Multistart bfs

In [ ]:
def findShortest(graph_nodes, graph_from, graph_to, ids, val):
    # solve here
    
    if len(graph_from) != len(graph_to):
        # Input is malformed
        return -1
        
    # Check if there is at least a pair of colors
    color_indicies = []
    for i, id in enumerate(ids):
        if id == val:
            color_indicies.append(i)

    # Short circuit if there aren't enough pairs
    if len(color_indicies) <= 1:
        return -1

    # Convert the graph to adjacency map
    adjacency_map = {}
    for i, from_node in enumerate(graph_from):
        to_list = [graph_to[i]]
        if from_node not in adjacency_map:
            adjacency_map[from_node] = to_list
        else:
            adjacency_map[from_node] += to_list
    
    for i, to_node in enumerate(graph_to):
        from_list = [graph_from[i]]
        if to_node not in adjacency_map:
            adjacency_map[to_node] = from_list
        else:
            adjacency_map[to_node] += from_list

    def bfs(start_node, target_color, adjacency_map, ids):
        q = deque([(start_node, 0)])
        visited = set([start_node])
        while len(q) != 0:
            node, cost_this_far = q.popleft()
            neighbors = adjacency_map[node]
            for n in neighbors:
                if n in visited:
                    continue
                if ids[n-1] == target_color:
                    return cost_this_far + 1
                visited.add(n)
                q.append((n, cost_this_far + 1))
        return -1
    
    
    print("Ids found", color_indicies)
    min_cost = float('inf')
    for i in range(len(color_indicies) - 1):
        index = color_indicies[i] + 1
        cost = bfs(index, val, adjacency_map, ids)
        print(f"planning from starting node {index}, cost found {cost}")
        min_cost = min(min_cost, cost)
    
    return min_cost

In [ ]:
from collections import deque, defaultdict

def findShortest(g_nodes, g_from, g_to, ids, val):
    # Build adjacency list
    graph = defaultdict(list)
    for u, v in zip(g_from, g_to):
        graph[u].append(v)
        graph[v].append(u)

    # Get all nodes with target color
    sources = [i + 1 for i, color in enumerate(ids) if color == val]
    if len(sources) < 2:
        return -1

    # Multi-source BFS
    queue = deque()
    visited = {}  # node -> (origin_source, distance_from_source)

    for src in sources:
        queue.append((src, src, 0))  # (current_node, source_node, distance)
        visited[src] = (src, 0)

    while queue:
        node, src, dist = queue.popleft()

        for neighbor in graph[node]:
            if neighbor not in visited:
                visited[neighbor] = (src, dist + 1)
                queue.append((neighbor, src, dist + 1))
            else:
                prev_src, prev_dist = visited[neighbor]
                # If this node was reached from another source, path found
                if prev_src != src:
                    return dist + prev_dist + 1

    return -1


# Depth First Search

In [ ]:
    def dfs(start, grid, filled_indicies, N, M):
        stack = [start]
        visited = set([start])
        filled_indicies.remove(start)
        
        while len(stack) != 0:
            node = stack.pop()
            neighbors = get_eight_neighbors(node, N, M)
            print("neighbors ", neighbors)
            for n in neighbors:
                if grid[n[0]][n[1]] != 1:
                    continue
                if n in visited:
                    continue
                visited.add(n)
                stack.append(n)
                filled_indicies.remove(n)
                
        print("visited ", visited)
        return len(visited)

# AStar and Dijkstra

In [ ]:
import heapq

def next_move(posx, posy, dimx, dimy, board):
    # We will perform multi-goal astar 
    goals = []
    for x in range(dimx):
        for y in range(dimy):
            if board[x][y] == 'd':
                goals.append((x, y))
    if len(goals) == 0:
        # Assume clean because there is no idle command
        print("FAILED")
        return

    
    goals_set = set(goals)
    
    open_set = []
    visited = {}
    
    start = (posx, posy)
    if start in goals_set:
        print("CLEAN")
        return
    
    def heuristic(cell, goals):
        x, y = cell
        min_dist = float('inf')
        for goal in goals:
            dist = abs(x - goal[0]) + abs(y - goal[1])
            min_dist = min(dist, min_dist)
        return min_dist
    
    
    def get_neighbors(cell, dimx, dimy):
        neighbors = []
        x, y = cell
        if x > 0:
            neighbors.append((x - 1, y))
        if y > 0:
            neighbors.append((x, y - 1))
        
        if x + 1 < dimx:
            neighbors.append((x + 1, y))
        
        if y + 1 < dimy:
            neighbors.append((x, y + 1))
            
        return neighbors
    def move_to_string(move):
        if move == (0, 1):
            return "RIGHT"
        if move == (0, -1):
            return "LEFT"
        if move == (-1, 0):
            return "UP"
        if move == (1, 0):
            return "DOWN"
    
    def get_move(cell, visited):
        while True:
            parent, _ = visited[cell]
            if visited[parent][0] is None:
                return (cell[0] - parent[0], cell[1] - parent[1])
            cell = parent
        return None
        
    heapq.heappush(open_set, (0 + heuristic(start, goals), start))
    visited[start] = (None, 0) # node: (parent, cost)
    
    while len(open_set) != 0:
        _, node = heapq.heappop(open_set)
        parent, cost = visited[node]
        neighbors = get_neighbors(node, dimx, dimy)
        for n in neighbors:
            nx, ny = n
            if n in visited:
                _, n_cost = visited[n]
                if cost + 1 < n_cost:
                    visited[n][1] = cost + 1
                    heapq.heappush(open_set, (cost + 1 + heuristic(n, goals), n))
                continue
            visited[n] = (node, cost + 1)
            if n in goals_set:
                print(move_to_string(get_move(n, visited)))
                return
            heapq.heappush(open_set, (cost + 1 + heuristic(n, goals), n))
    
    print("FAILED")

if __name__ == "__main__":
    pos = [int(i) for i in input().strip().split()]
    dim = [int(i) for i in input().strip().split()]
    board = [[j for j in input().strip()] for i in range(dim[0])]
    next_move(pos[0], pos[1], dim[0], dim[1], board)

# Number of islands

In [ ]:
def numIslands(self, grid: List[List[str]]) -> int:

        if not grid:
            return 0
        
        rows, cols = len(grid), len(grid[0])
        islands = 0
        directions = [(-1,0), (1,0), (0,-1), (0,1)]  # Up, Down, Left, Right

        for r in range(rows):
            for c in range(cols):
                if grid[r][c] == '1':
                    islands += 1
                    grid[r][c] = '0'  # mark as visited immediately
                    queue = deque([(r, c)])

                    while queue:
                        x, y = queue.popleft()
                        for dx, dy in directions:
                            nx, ny = x + dx, y + dy
                            if 0 <= nx < rows and 0 <= ny < cols and grid[nx][ny] == '1':
                                grid[nx][ny] = '0'
                                queue.append((nx, ny))
        return islands
        

# Binary Search

In [ ]:
#!/bin/python3

import math
import os
import random
import re
import sys



#
# Complete the 'findFirstOccurrence' function below.
#
# The function is expected to return an INTEGER.
# The function accepts following parameters:
#  1. INTEGER_ARRAY nums
#  2. INTEGER target
#

def findFirstOccurrence(nums, target):
    # Write your code here
    # binary search
    
    def recurse(nums, target, min_index, max_index):
        if min_index > max_index:
            return -1
        index = min_index + (max_index - min_index) // 2
        if nums[index] == target:
            # Check if it's the first occurrence
            if index == 0 or nums[index - 1] != target:
                return index
            else:
                return recurse(nums, target, min_index, index - 1)
                
        if nums[index] < target:
            return recurse(nums, target, index + 1, max_index)
        else:
            return recurse(nums, target, min_index, index - 1)
            
    if len(nums) == 0:
        return -1
    return recurse(nums, target, 0, len(nums) - 1)
    

if __name__ == '__main__':
    nums_count = int(input().strip())

    nums = []

    for _ in range(nums_count):
        nums_item = int(input().strip())
        nums.append(nums_item)

    target = int(input().strip())

    result = findFirstOccurrence(nums, target)

    print(result)
